In [409]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [410]:
# Convert 'DateTime' column to datetime format

#csv = "../data/Sprint 3/BK27276M Lijn 1 23600 records.csv"
csv = "../data/Sprint 3/BK32348M Lijn 3 21379 records.csv"
#csv = "../data/Sprint 3/BK33725M Lijn 1 2496 records.csv"
#csv = "../data/Sprint 3/BK35553M Lijn 3 3489 records.csv"

def custom_date_parser(date):
    formats = ["%m/%d/%Y %I:%M:%S %p", "%m-%d-%Y %H:%M"]
    for fmt in formats:
        try:
            return datetime.strptime(date, fmt)
        except ValueError:
            pass
    raise ValueError("No valid date format found")

data = pd.read_csv(csv, sep=';', decimal=",", date_parser=custom_date_parser, parse_dates=[0])
data.columns

C:\Users\Johnny\AppData\Local\Temp\ipykernel_29248\475827991.py:17: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv(csv, sep=';', decimal=",", date_parser=custom_date_parser, parse_dates=[0])


Index(['DateTime', 'EL02_Process_Recipe_Working,MESQuality,Density_Max',
       'EL02_Process_Recipe_Working,Extruder,AltBom',
       'EL02_Process_Recipe_Working,Extruder,DietCode',
       'EL02_Process_Recipe_Working,Extruder,ProdOrderNum',
       'EL02_Process_Recipe_Working,MESQuality,Density_Min',
       'EL02_Process_Recipe_Working,MESQuality,Density_Target',
       'EL02_Dry_Feed_Rate_PID,OUT', 'EL02_Dry_Feed_Rate_PID,PV',
       'EL02_Dry_Feed_Rate_PID,SP'],
      dtype='object')

In [411]:
def clean_column_name(name: str):
	return name.split(",")[-1]

data = data.rename(columns=clean_column_name)
data.columns

Index(['DateTime', 'Density_Max', 'AltBom', 'DietCode', 'ProdOrderNum',
       'Density_Min', 'Density_Target', 'OUT', 'PV', 'SP'],
      dtype='object')

In [412]:
data.head()

,DateTime,Density_Max,AltBom,DietCode,ProdOrderNum,Density_Min,Density_Target,OUT,PV,SP
0,2023-11-22 18:09:29,376,011-00,BK32348M,9871289,312,344,0.0,8.940067,0
1,2023-11-22 18:09:34,376,011-00,BK32348M,9871289,312,344,0.0,34.135712,0
2,2023-11-22 18:09:39,376,011-00,BK32348M,9871289,312,344,0.0,116.290634,0
3,2023-11-22 18:09:44,376,011-00,BK32348M,9871289,312,344,0.0,148.333359,0
4,2023-11-22 18:09:49,376,011-00,BK32348M,9871289,312,344,0.0,53.032440,0


In [413]:
data = data.dropna()

In [414]:
data["Year"] = data["DateTime"].dt.year
data["Month"] = data["DateTime"].dt.month
data["Day"] = data["DateTime"].dt.day
data["Hour"] = data["DateTime"].dt.hour
data["Minute"] = data["DateTime"].dt.minute
data = data.drop(columns=["DateTime", "DietCode", "AltBom", "Density_Max", "ProdOrderNum", "Density_Min", "Density_Target"])
data.head()

,OUT,PV,SP,Year,Month,Day,Hour,Minute
0,0.0,8.940067,0,2023,11,22,18,9
1,0.0,34.135712,0,2023,11,22,18,9
2,0.0,116.290634,0,2023,11,22,18,9
3,0.0,148.333359,0,2023,11,22,18,9
4,0.0,53.032440,0,2023,11,22,18,9


In [415]:
data.dtypes

OUT       float64
PV        float64
SP          int64
Year        int32
Month       int32
Day         int32
Hour        int32
Minute      int32
dtype: object

In [416]:
# rows = len(data)
len(data[data["OUT"] == 0])
# removed = rows - len(data)
# print(f"Removed {removed} rows")

123

In [417]:
# Calculate IQR and potential outliers
Q1 = data['OUT'].quantile(0.25)
Q3 = data['OUT'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = data[(data['OUT'] < lower_bound) | (data['OUT'] > upper_bound)]
print("Number of outliers:", len(outliers))

data = data[(data['OUT'] >= lower_bound) & (data['OUT'] <= upper_bound)]
data.head()

Number of outliers: 499


,OUT,PV,SP,Year,Month,Day,Hour,Minute
38,44.0,55.653656,7000,2023,11,22,18,12
39,44.0,58.015755,7000,2023,11,22,18,12
40,44.0,284.826355,7000,2023,11,22,18,12
41,44.0,3362.511719,7000,2023,11,22,18,12
42,44.0,6778.325195,7000,2023,11,22,18,12


In [418]:
# Calculate IQR and potential outliers
Q1 = data['PV'].quantile(0.05)
Q3 = data['PV'].quantile(0.95)
IQR = Q3 - Q1

lower_bound = Q1 - 1.05 * IQR
upper_bound = Q3 + 1.05 * IQR

outliers = data[(data['PV'] < lower_bound) | (data['PV'] > upper_bound)]
print("Number of outliers:", len(outliers))

outliers.head(n = 100)

Number of outliers: 14


,OUT,PV,SP,Year,Month,Day,Hour,Minute
38,44.00000,55.653656,7000,2023,11,22,18,12
39,44.00000,58.015755,7000,2023,11,22,18,12
40,44.00000,284.826355,7000,2023,11,22,18,12
41,44.00000,3362.511719,7000,2023,11,22,18,12
6032,40.54129,7760.645508,7000,2023,11,23,2,32
9540,44.00000,77.699844,0,2023,11,30,5,42
9541,44.00000,135.187378,7000,2023,11,30,5,43
9542,44.00000,148.582519,7000,2023,11,30,5,43
9543,44.00000,1084.170044,7000,2023,11,30,5,43
9544,44.00000,5074.100586,7000,2023,11,30,5,43


In [419]:
data = data[(data['PV'] >= lower_bound) & (data['PV'] <= upper_bound)]
data.head()

,OUT,PV,SP,Year,Month,Day,Hour,Minute
42,44.000000,6778.325195,7000,2023,11,22,18,12
43,44.000000,7123.797852,7000,2023,11,22,18,13
44,43.065243,7330.137695,7000,2023,11,22,18,13
45,43.057755,7197.839844,7000,2023,11,22,18,13
46,42.700405,7231.707031,7000,2023,11,22,18,13


In [420]:
data.columns

Index(['OUT', 'PV', 'SP', 'Year', 'Month', 'Day', 'Hour', 'Minute'], dtype='object')

In [421]:
data.to_csv(csv.replace(".csv", ".etl.csv"), sep=";", decimal=",", index=False)